In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import chart_studio.plotly as py 
import plotly.graph_objs as go #importing graphical objects
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [4]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="marinelin")

Location(Carlsbad, San Diego County, California, United States of America, (33.1580933, -117.3505966, 0.0))

In [53]:
job = job.dropna()
job_us = job[job['location'].str.contains('US')]

In [54]:
job_us['geo_location'] = job_us['location'].apply(lambda x: geolocator.geocode(x,addressdetails=True))

In [57]:
state=[]
for items in job_us['geo_location']:
    if 'state' in items.raw['address'].keys():
        state.append(items.raw['address']['state'])
    else:
        state.append('None')

In [58]:
job_us['state']=state

In [60]:
job_us['total_posting'] = job_us[['job_id','state']].groupby('state').transform('count')

In [61]:
job_us['fradulent_count']=job_us[['job_id','state','fraudulent']].groupby(['state','fraudulent']).transform('count')

In [65]:
job_us['percentage_f']=job_us['fradulent_count']/job_us['total_posting']

In [84]:
job_us = job_us[job_us['total_posting']>15] #select state with more than 10 posting 

In [85]:
job_us['log_lat']=job_us['geo_location'].apply(lambda x: x[-1])
job_us['count'] = job_us[['job_id','log_lat','fraudulent']].groupby(['log_lat','fraudulent']).transform('count')
#df1 = job_us[['log_lat','fraudulent','count']]
job_us['long'] = job_us['log_lat'].apply(lambda x: x[0])
job_us['lat'] = job_us['log_lat'].apply(lambda x: x[1])
#df1 = df1.drop(['log_lat'],axis=1)

In [86]:
import us

In [87]:
job_us = job_us[job_us['state']!='None']

In [88]:
state_abr=[]
for items in job_us['state']:
    if us.states.lookup(items) is not None:
        state_abr.append(us.states.lookup(items).abbr)
    else:
        state_abr.append('None')

In [89]:
job_us['state_abr'] = state_abr

In [90]:
job_us = job_us[job_us['state_abr'] !='None']

In [93]:
data = dict(type='choropleth',
            locations = job_us['state_abr'],
            locationmode = 'USA-states',
            colorscale = 'Reds',
            text = job_us['percentage_f'],
            z = job_us['percentage_f'],
            colorbar = {'title':"percentage of fraudulent posting"},
            marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ))
            )

layout = dict(
        title = 'US percentage of fraudulent posting by state',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
        ),
    )
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)